In [37]:
import quandl
import os
import pandas as pd
import pickle
import bs4 as bs
import requests
from sklearn import preprocessing

In [38]:
quandl.ApiConfig.api_key='4sg3wyxBDeENg3R3W3qt'
startdate="2014-04-22"
enddate="2021-04-19"

In [39]:
def nifty_50_list():
    resp = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    print(f"soup:{soup}")

In [40]:
nifty_50_list()


soup:<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>NIFTY 50 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6e9a5cef-8379-4200-b9a5-e025457a65b4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"NIFTY_50","wgTitle":"NIFTY 50","wgCurRevisionId":1021568465,"wgRevisionId":1021568465,"wgArticleId":1328813,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: generic title","Articles with short description","Short description is different from Wikidata","Use Indian English from September 2020","All Wikipedia articles writ

In [41]:
def nifty_50_list():
    resp = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    #Goto wiki page and do inspect element
    # Find all the table tags whose class is wikitable sortable, insie that there is 'tbody' tag
    # and extract all the data from there 
    table = soup.find('table', {'class': 'wikitable sortable'},'tbody')
    print(table)
nifty_50_list()

<table class="wikitable sortable" id="constituents" style="font-size:85%">
<tbody><tr>
<th>Company Name</th>
<th>Symbol</th>
<th>Sector
</th></tr>
<tr>
<td><a href="/wiki/Adani_Ports_%26_SEZ" title="Adani Ports &amp; SEZ">Adani Ports</a></td>
<td>ADANIPORTS.NS</td>
<td>Infrastructure
</td></tr>
<tr>
<td><a href="/wiki/Asian_Paints" title="Asian Paints">Asian Paints</a></td>
<td>ASIANPAINT.NS</td>
<td>Consumer Goods
</td></tr>
<tr>
<td><a href="/wiki/Axis_Bank" title="Axis Bank">Axis Bank</a></td>
<td>AXISBANK.NS</td>
<td>Banking
</td></tr>
<tr>
<td><a href="/wiki/Bajaj_Auto" title="Bajaj Auto">Bajaj Auto</a></td>
<td>BAJAJ-AUTO.NS</td>
<td>Automobile
</td></tr>
<tr>
<td><a href="/wiki/Bajaj_Finance" title="Bajaj Finance">Bajaj Finance</a></td>
<td>BAJFINANCE.NS</td>
<td>Financial Services
</td></tr>
<tr>
<td><a href="/wiki/Bajaj_Finserv" title="Bajaj Finserv">Bajaj Finserv</a></td>
<td>BAJAJFINSV.NS</td>
<td>Financial Services
</td></tr>
<tr>
<td><a href="/wiki/Bharti_Airtel" title="Bh

In [42]:
def nifty_50_list():
    resp = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    #Goto wiki page and do inspect element
    # Find all the table tags whose class is wikitable sortable, insie that there is 'tbody' tag
    # and extract all the data from there 
    table = soup.find('table', {'class': 'wikitable sortable'},'tbody')

    tickers = []
    for row in table.findAll('tr')[1:]: # 0th <tr> tag contains Company,Sysmbol,Sector.. Not needed
        ticker = row.findAll('td')[1].text  #First <td> tag data
        print(ticker.split('.')[0])
nifty_50_list()

ADANIPORTS
ASIANPAINT
AXISBANK
BAJAJ-AUTO
BAJFINANCE
BAJAJFINSV
BHARTIARTL
BPCL
BRITANNIA
CIPLA
COALINDIA
DIVISLAB
DRREDDY
EICHERMOT
GRASIM
HCLTECH
HDFC
HDFCBANK
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDUNILVR
ICICIBANK
INDUSINDBK
INFY
IOC
ITC
JSWSTEEL
KOTAKBANK
LT
M&M
MARUTI
NESTLEIND
NTPC
ONGC
POWERGRID
RELIANCE
SBIN
SBILIFE
SHREECEM
SUNPHARMA
TATAMOTORS
TATASTEEL
TCS
TATACONSUM
TECHM
TITAN
ULTRACEMCO
UPL
WIPRO


In [7]:
#ticker symbol - The abbreviation representing companies in the stock market. Example: Google --> GOOGL, Apple---> AAPL

In [43]:
def nifty_50_list():
    resp = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs.BeautifulSoup(resp.text, 'lxml')

    table = soup.find('table', {'class': 'wikitable sortable'},'tbody')

    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        # print(f"ticker{ticker}")
        tickers.append(ticker.split('.')[0])

    with open("nifty50_list.pickle","wb") as f:
        pickle.dump(tickers,f)
    
    #Adding it manually since ticker name obtained from Wikipedia contains a hypen whereas quandl code expects an underscore
    tickers = list(map(lambda x: x.replace("BAJAJ-AUTO","BAJAJ_AUTO"), tickers))
    #Adding it manually since quandl code is different than the ticker symbol obtained from Wiki which is M&M
    tickers = list(map(lambda x: x.replace("M&M","MM"), tickers))
    #Fetching data for NIFTY50 index whose price we want to PREDICT
    tickers.append('NIFTY_50')
    # We will remove below ones since data ins't available
    tickers.remove('VEDL')
    tickers.remove('UPL')
#     tickers.remove('IBULHSGFIN')
    return tickers

In [44]:
#function to scrap NIFTY50 list from WIKI only if not already obtained
def get_nifty50_list(scrap=False):
    if scrap:
        tickers=nifty_50_list()
    else:
        with open("nifty50_list.pickle","rb") as f:
            tickers=pickle.load(f)
    return tickers

In [45]:

def getStockdataFromQuandl(ticker):
    quandl_code="NSE/"+ticker
    try:
        if not os.path.exists(f'stock_nifty/{ticker}.csv'):
          data=quandl.get(quandl_code,start_date=startdate,end_date=enddate)
          data.to_csv(f'stock_nifty/{ticker}.csv')
        else:
            print(f"stock data for {ticker} already exists")
    except quandl.errors.quandl_error.NotFoundError as e:
        print(ticker)
        print(str(e))

In [46]:
def load():
    tickers=get_nifty50_list(True)
    df=pd.DataFrame()
    for ticker in tickers:
        getStockdataFromQuandl(ticker) # Store them in csv
        try:
            data=pd.read_csv(f'stock_data/{ticker}.csv')
            
            # Due to column name mismatches we are renaming it from stock_data/...csv to our dataframe
            if(ticker == "NIFTY_50"):
                data.rename(columns={'Close':f"{ticker}_Close",'Shares Traded':f"{ticker}_Volume"},inplace=True)
            else:
                data.rename(columns={'Close':f"{ticker}_Close",'Total Trade Quantity':f"{ticker}_Volume"},inplace=True)
            
            df=pd.concat([df,data[f'{ticker}_Volume'],data[f'{ticker}_Close']],axis=1)
        except Exception as e:
            print(f"couldn't find {ticker}")
            print(str(e))
    df.to_csv('nifty50_closingprices.csv')
    df.dropna(inplace=True)
    return df

In [47]:
load()

ValueError: list.remove(x): x not in list